In [11]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split 
import numpy as np
import pandas as pd
import time
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from scipy import stats
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
#from bert_serving.client import BertClient
from sklearn.multiclass import OneVsOneClassifier
#import gensim
#from gensim.utils import simple_preprocess
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#import gensim.corpora as corpora
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.sparse import hstack
#import gensim.downloader as api
#from gensim.corpora import Dictionary
#from gensim.similarities import SparseTermSimilarityMatrix
#from gensim.similarities import WordEmbeddingSimilarityIndex
#from gensim.models import TfidfModel
#from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def vectorization_TF_IDF(DATASET, text_field_name, label_name):
    
    X_train, X_test, y_train, y_test = train_test_split( DATASET[text_field_name] ,
                                                    DATASET[label_name], 
                                                    train_size=0.80, random_state=28)

    encoder = preprocessing.LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.fit_transform(y_test)
    
    return X_train, X_test, y_train, y_test

def saving_results(results, path, file_name):

    results.to_csv(path + file_name,  index = False, sep=";")
    
def train_model(classifier, X_train_v, X_test_v, y_train, y_test):
    ini = time.time()
    classifier.fit(X_train_v, y_train)
    predictions = classifier.predict(X_test_v)
    fim = time.time()
    
    timee = fim-ini
    acuracia = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, predictions)
    recall = recall_score(y_test, predictions, average='weighted')
    f1_score = metrics.f1_score(y_test, predictions, average='weighted')
    print("Time: " + str(timee))
    print("Acurácia: " + str(acuracia))
    print("Precision: " + str(precision))
    print("Recal: " + str(recall))
    print("f1_score: " + str(f1_score))
    
    return timee , acuracia, precision, recall, f1_score, matriz_confusao

def get_tests_result(X_train_v, X_test_v, y_train, y_test):
    
    all_res = []
    ### MLPClassifier
    print( "MLPClassifier")
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(70, ), random_state=1, verbose=True)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["MLPClassifier: ", timee , acuracia, precision, recall, f1_score])
    
    
    ### AdaBoostClassifier
    print( "AdaBoostClassifier: ")
    clf = AdaBoostClassifier(n_estimators= 50, learning_rate=1)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["AdaBoostClassifier: ", timee , acuracia, precision, recall, f1_score])
    
    ### Voting_LR3_SVC1_ETC2
    print( "Voting_LR3_SVC1_ETC2: ")
    ini = time.time()
    clf2 = LogisticRegression(random_state=0)
    clf5 = SVC(kernel='rbf', probability=True)
    clf6 = ExtraTreesClassifier(n_estimators=100, random_state=0)
    eclf = VotingClassifier(estimators=[ ('lr', clf2), ('svc', clf5), ('etc', clf6)], voting='soft', weights=[3, 1, 2])
    clf2 = clf2.fit(X_train_v, y_train)
    clf5 = clf5.fit(X_train_v, y_train)
    clf6 = clf6.fit(X_train_v, y_train)
    eclf = eclf.fit(X_train_v, y_train)
    #Y_previsto_vc1 = eclf.predict(X_test_v.toarray())
    Y_previsto_vc1 = eclf.predict(X_test_v)
    fim = time.time()
    
    timee = fim-ini
    acuracia = accuracy_score(y_test, Y_previsto_vc1)
    precision = precision_score(y_test, Y_previsto_vc1, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, Y_previsto_vc1)
    recall = recall_score(y_test, Y_previsto_vc1, average='weighted')
    f1_score = metrics.f1_score(y_test, Y_previsto_vc1, average='weighted')
    print("Time: " + str(timee))
    print("Acurácia: " + str(acuracia))
    print("Precision: " + str(precision))
    print("Recal: " + str(recall))
    print("f1_score: " + str(f1_score))
    print(matriz_confusao)
    all_res.append(["Voting_LR3_SVC1_ETC2: ", timee , acuracia, precision, recall, f1_score])
    
    ### DecisionTreeClassifier
    print( "DecisionTreeClassifier: ") 
    clf = DecisionTreeClassifier(random_state=0)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)    
    print(matriz_confusao)
    all_res.append(["DecisionTreeClassifier: ", timee , acuracia, precision, recall, f1_score])
    
    ### RandomForestClassifier
    print( "RandomForestClassifier: " )
    clf = RandomForestClassifier(n_estimators=50, random_state=1)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test) 
    print(matriz_confusao)
    all_res.append(["RandomForestClassifier: ", timee , acuracia, precision, recall, f1_score])
    
    ### ExtraTreesClassifier
    print( "ExtraTreesClassifier: ")
    clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["ExtraTreesClassifier: ", timee , acuracia, precision, recall, f1_score])
    
    ### LogisticRegression
    print( "LogisticRegression: ") 
    clf = LogisticRegression(random_state=0)
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["LogisticRegression: ",  timee , acuracia, precision, recall, f1_score])
    
    ### svm
    print( "svm: ") 
    clf = svm.SVC()
    timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    print(matriz_confusao)
    all_res.append(["svm: ",  timee , acuracia, precision, recall, f1_score])
    
    ### Stacking_scikit
    print( "Stacking_scikit: ")
    clf1 = RandomForestClassifier(n_estimators=50, random_state=1)
    clf2 = LogisticRegression(random_state=0)
    clf3 = SVC()
    clf4 = ExtraTreesClassifier(n_estimators=100, random_state=0)
    estimators = [('rf', clf1), ('lr', clf2), ('svc', clf3), ('etc', clf4)]
    ini = time.time()
    clf = StackingClassifier( estimators=estimators, final_estimator=LogisticRegression(), n_jobs=None)
    clf.fit(X_train_v, y_train)
    Y_Previsto_stacking = clf.predict(X_test_v)
    fim = time.time()
    
    timee = fim-ini
    acuracia = accuracy_score(y_test, Y_Previsto_stacking)
    precision = precision_score(y_test, Y_Previsto_stacking, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, Y_Previsto_stacking)
    recall = recall_score(y_test, Y_Previsto_stacking, average='weighted')
    f1_score = metrics.f1_score(y_test, Y_Previsto_stacking, average='weighted')
    print("Time: " + str(timee))
    print("Acurácia: " + str(acuracia))
    print("Precision: " + str(precision))
    print("Recal: " + str(recall))
    print("f1_score: " + str(f1_score))
    print(matriz_confusao)
    all_res.append(["Stacking_scikit: ",  timee , acuracia, precision, recall, f1_score])
    
    ### XGBClassifier
    #print( "XGBClassifier: ") 
    #clf = XGBClassifier(eval_metric='mlogloss' )
    #timee , acuracia, precision, recall, f1_score, matriz_confusao = train_model(clf, X_train_v, X_test_v, y_train, y_test)
    #print(matriz_confusao)
    #all_res.append(["XGBClassifier: ",  timee , acuracia, precision, recall, f1_score])
    
    ### OneVsRestClassifier_RF
    print( "OneVsRest_RF: ") 
    clf = RandomForestClassifier(n_estimators=50, random_state=1)
    ovr = OneVsRestClassifier(clf)
    ini = time.time()
    ovr.fit(X_train_v, y_train)
    Y_Previsto_OVR_RF = ovr.predict(X_test_v)
    fim = time.time()
    
    
    timee = fim-ini
    acuracia = accuracy_score(y_test, Y_Previsto_OVR_RF)
    precision = precision_score(y_test, Y_Previsto_OVR_RF, average='weighted')
    matriz_confusao =  confusion_matrix(y_test, Y_Previsto_OVR_RF)
    recall = recall_score(y_test, Y_Previsto_OVR_RF, average='weighted')
    f1_score = metrics.f1_score(y_test, Y_Previsto_OVR_RF, average='weighted')
    print("Time: " + str(timee))
    print("Acurácia: " + str(acuracia))
    print("Precision: " + str(precision))
    print("Recal: " + str(recall))
    print("f1_score: " + str(f1_score))
    print(matriz_confusao)
    all_res.append(["OvR_RF: ",  timee , acuracia, precision, recall, f1_score])
    
    return pd.DataFrame( all_res, columns=["ALGORITHM","TIME","ACCURACY","PRECISION","RECALL", "F1_SCORE"])

feat = ['Idade', 'Peso Kg', 'Cor da pele', 'ACTN3',
       'AGT', 'ECA', 'Score Força%', 'Score Resistência %',
       'Pré CK (U/L)', 'Pós CK (U/L)', 'Pré CK MB U/L', 'Pós CK MB U/L',
       'Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)', 'Pré LDH (U/L)',
       'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L',
       'Osmolalidade sérica PRÉ BD4(mOsm/kg): 1,86(Na + K) + 1,15(Gli /18) + (Ureia/6) + 14\t\t\t',
       'Osmolalidade sérica PÓS BD4(mOsm/kg): 1,86(Na + K) + 1,15(Gli /18) + (Ureia/6) + 14\t\t\t',
       'Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %',
       'Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %', 'massa magra', 'MCT KG', 'ESTATURA', 'vo2max estimado',
       'Handgrip     Dom.', 'Handgrip Cont.', '% gordura']

label_rabdomiolise = "Rabdomiólise"
label_lesao = "Lesão Renal Aguda"




In [163]:
#path = "D:/OneDrive/PC Venicius/ESAO/TCC/"
path = "C:/Users/venic/OneDrive/PC Venicius/ESAO/TCC/"
file = 'dados.xlsx'
DATASET = pd.read_excel(path + file) 
DATASET

,Ano,Missão,Aluno,Idade,Peso Kg,Cor da pele,ACTN3,AGT,ECA,BDKRB2,...,Pós BASO %,Lesão Renal Aguda,Rabdomiólise,massa magra,MCT KG,ESTATURA,vo2max estimado,Handgrip Dom.,Handgrip Cont.,% gordura
0,2018,CIASC,3150,33,88,1,XX,MT,ID,"(-9,-9)",...,0.1,0.0,False,75.52,84.3,181.5,50.85,35.0,37.0,10.05
1,2018,CIASC,3155,26,74,1,RR,MT,II,"(+9,-9)",...,0.2,0.0,True,69.49,75.9,171.0,51.98,42.0,43.0,6.08
2,2018,CIASC,3156,26,76,1,RX,MM,ID,"(+9,+9)",...,0.3,0.0,True,72.07,79.2,174.0,52.42,49.5,49.5,8.72
3,2018,CIASC,3159,25,84,1,RR,TT,DD,"(+9,-9)",...,0.9,0.0,True,75.26,84.3,178.0,50.68,60.0,57.0,10.61
4,2018,CIASC,3165,33,82,1,RR,MT,II,"(+9,-9)",...,0.1,0.0,True,66.26,79.6,182.0,47.64,51.0,49.0,14.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,2023,Manaus,3340,29,NaN,2,NaN,NaN,NaN,NaN,...,NaN,0.0,False,"76,40",82.8,182.0,"53,54",56.0,59.0,"7,73"
463,2023,Manaus,3343,27,NaN,2,NaN,NaN,NaN,NaN,...,NaN,0.0,False,"65,80",75.1,170.0,-,48.0,60.0,"12,39"
464,2023,Manaus,3347,28,NaN,2,NaN,NaN,NaN,NaN,...,NaN,0.0,False,"74,27",82.7,179.5,"50,56",58.0,52.0,"10,20"
465,2023,Manaus,3348,29,NaN,2,NaN,NaN,NaN,NaN,...,NaN,0.0,False,"70,18",80.5,181.0,"51,80",60.0,50.0,"12,82"


In [92]:
DATASET.columns

Index(['Ano', 'Missão', 'Aluno', 'Idade', 'Peso Kg', 'Cor da pele', 'ACTN3',
       'AGT', 'ECA', 'BDKRB2', 'Score Força%', 'Score Resistência %',
       'Pré CK (U/L)', 'Pós CK (U/L)', 'Pré CK MB U/L', 'Pós CK MB U/L',
       'Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)', 'Pré LDH (U/L)',
       'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose 

### Teste 1 -  'Pré CK (U/L)', 'Pós CK (U/L)', "Rabdomiólise"

In [84]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré CK (U/L)', 'Pós CK (U/L)']
col_to_be_encoded = ["Rabdomiólise"]

col_df = ['Pré CK (U/L)', 'Pós CK (U/L)', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)
    
for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_feature: 
    df[d] = df[d].astype(float)

df

,Pré CK (U/L),Pós CK (U/L),Rabdomiólise
0,337.0,664.0,0
1,687.0,2246.0,1
2,841.0,4616.0,1
3,363.0,2611.0,1
4,192.0,1111.0,1
...,...,...,...
423,223.0,1570.0,1
424,340.0,11324.0,1
425,312.0,4688.0,1
426,109.0,6710.0,1


In [86]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label], 
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

MLPClassifier
Time: 0.32726168632507324
Acurácia: 0.6666666666666666
Precision: 0.611764705882353
Recal: 0.6666666666666666
f1_score: 0.6171648987463838
[[ 4 20]
 [ 6 48]]
AdaBoostClassifier: 
Time: 0.07812786102294922
Acurácia: 1.0
Precision: 1.0
Recal: 1.0
f1_score: 1.0
[[24  0]
 [ 0 54]]
Voting_LR3_SVC1_ETC2: 
Time: 0.2238614559173584
Acurácia: 0.9487179487179487
Precision: 0.9487179487179487
Recal: 0.9487179487179487
f1_score: 0.9487179487179487
[[22  2]
 [ 2 52]]
DecisionTreeClassifier: 
Time: 0.0
Acurácia: 1.0
Precision: 1.0
Recal: 1.0
f1_score: 1.0
[[24  0]
 [ 0 54]]
RandomForestClassifier: 
Time: 0.06308269500732422
Acurácia: 1.0
Precision: 1.0
Recal: 1.0
f1_score: 1.0
[[24  0]
 [ 0 54]]
ExtraTreesClassifier: 
Time: 0.09355425834655762
Acurácia: 0.9615384615384616
Precision: 0.9623222060957911
Recal: 0.9615384615384616
f1_score: 0.9617511994013997
[[23  1]
 [ 2 52]]
LogisticRegression: 
Time: 0.017281532287597656
Acurácia: 0.9487179487179487
Precision: 0.9487179487179487
Recal:

,ALGORITHM,TIME,ACCURACY,PRECISION,RECALL,F1_SCORE
0,MLPClassifier:,0.327262,0.666667,0.611765,0.666667,0.617165
1,AdaBoostClassifier:,0.078128,1.000000,1.000000,1.000000,1.000000
2,Voting_LR3_SVC1_ETC2:,0.223861,0.948718,0.948718,0.948718,0.948718
3,DecisionTreeClassifier:,0.000000,1.000000,1.000000,1.000000,1.000000
4,RandomForestClassifier:,0.063083,1.000000,1.000000,1.000000,1.000000
5,ExtraTreesClassifier:,0.093554,0.961538,0.962322,0.961538,0.961751
6,LogisticRegression:,0.017282,0.948718,0.948718,0.948718,0.948718
7,svm:,0.006038,0.910256,0.930521,0.910256,0.912857
8,Stacking_scikit:,1.000134,0.987179,0.987692,0.987179,0.987250
9,XGBClassifier:,0.032696,1.000000,1.000000,1.000000,1.000000


# Teste 2 - Pré CK MB e Pós CK MB

In [88]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré CK MB U/L', 'Pós CK MB U/L']
col_to_be_encoded = ["Rabdomiólise"]

col_df = ['Pré CK MB U/L', 'Pós CK MB U/L', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)
    
for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_feature: 
    df[d] = df[d].astype(float)

df

,Pré CK MB U/L,Pós CK MB U/L,Rabdomiólise
0,3.0,16.0,0
1,6.0,38.0,1
2,43.0,71.0,1
3,5.0,53.0,1
4,3.0,15.0,1
...,...,...,...
339,20.0,153.0,1
340,20.0,45.0,0
341,18.0,57.0,0
342,19.0,76.0,1


In [90]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label], 
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

MLPClassifier
Time: 0.1286003589630127
Acurácia: 0.5942028985507246
Precision: 0.6169366297243535
Recal: 0.5942028985507246
f1_score: 0.5695211428275129
[[12 22]
 [ 6 29]]
AdaBoostClassifier: 
Time: 0.09389138221740723
Acurácia: 0.6231884057971014
Precision: 0.6662564943943123
Recal: 0.6231884057971014
f1_score: 0.5938998682476944
[[12 22]
 [ 4 31]]
Voting_LR3_SVC1_ETC2: 
Time: 0.22892260551452637
Acurácia: 0.6376811594202898
Precision: 0.6391867410889149
Recal: 0.6376811594202898
f1_score: 0.6372241669789449
[[23 11]
 [14 21]]
DecisionTreeClassifier: 
Time: 0.003974437713623047
Acurácia: 0.6086956521739131
Precision: 0.6313501144164759
Recal: 0.6086956521739131
f1_score: 0.5878492496260194
[[13 21]
 [ 6 29]]
RandomForestClassifier: 
Time: 0.06251263618469238
Acurácia: 0.5942028985507246
Precision: 0.6169366297243535
Recal: 0.5942028985507246
f1_score: 0.5695211428275129
[[12 22]
 [ 6 29]]
ExtraTreesClassifier: 
Time: 0.09374165534973145
Acurácia: 0.6231884057971014
Precision: 0.666256

,ALGORITHM,TIME,ACCURACY,PRECISION,RECALL,F1_SCORE
0,MLPClassifier:,0.128600,0.594203,0.616937,0.594203,0.569521
1,AdaBoostClassifier:,0.093891,0.623188,0.666256,0.623188,0.593900
2,Voting_LR3_SVC1_ETC2:,0.228923,0.637681,0.639187,0.637681,0.637224
3,DecisionTreeClassifier:,0.003974,0.608696,0.631350,0.608696,0.587849
4,RandomForestClassifier:,0.062513,0.594203,0.616937,0.594203,0.569521
5,ExtraTreesClassifier:,0.093742,0.623188,0.666256,0.623188,0.593900
6,LogisticRegression:,0.000000,0.623188,0.623188,0.623188,0.623188
7,svm:,0.002893,0.652174,0.674858,0.652174,0.642279
8,Stacking_scikit:,0.955424,0.623188,0.625501,0.623188,0.622237
9,XGBClassifier:,0.062981,0.594203,0.616937,0.594203,0.569521


# Teste 3 - 'ACTN3',  'AGT', 'ECA', 'BDKRB2', 'Score Força%', 'Score Resistência %,

In [95]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['ACTN3', 'AGT', 'ECA', 'BDKRB2', 'Score Força%', 'Score Resistência %']
col_to_be_encoded = ['ACTN3', 'AGT', 'ECA', 'BDKRB2', "Rabdomiólise"]
col_to_float = ['Score Força%', 'Score Resistência %']

col_df = ['ACTN3', 'AGT', 'ECA', 'BDKRB2', 'Score Força%', 'Score Resistência %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)
    
for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float: 
    df[d] = df[d].astype(float)

df

,ACTN3,AGT,ECA,BDKRB2,Score Força%,Score Resistência %,Rabdomiólise
0,3,1,1,6,25.0,75.0,0
1,0,1,2,3,50.0,50.0,1
2,1,0,1,2,50.0,50.0,1
4,0,1,2,3,50.0,50.0,1
5,3,1,1,6,100.0,0.0,1
...,...,...,...,...,...,...,...
338,3,2,0,3,62.5,37.5,1
339,0,0,0,6,50.0,50.0,1
340,3,2,1,6,37.5,62.5,0
342,0,1,1,6,50.0,50.0,1


In [97]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label], 
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

MLPClassifier
Time: 0.30646419525146484
Acurácia: 0.48148148148148145
Precision: 0.46063710769593125
Recal: 0.48148148148148145
f1_score: 0.4654684095860566
[[ 6 17]
 [11 20]]
AdaBoostClassifier: 
Time: 0.07485246658325195
Acurácia: 0.5
Precision: 0.48456790123456783
Recal: 0.5
f1_score: 0.4881689115398617
[[ 7 16]
 [11 20]]
Voting_LR3_SVC1_ETC2: 
Time: 0.2365741729736328
Acurácia: 0.4444444444444444
Precision: 0.3711533944092084
Recal: 0.4444444444444444
f1_score: 0.39145027380321507
[[ 2 21]
 [ 9 22]]
DecisionTreeClassifier: 
Time: 0.003602743148803711
Acurácia: 0.4444444444444444
Precision: 0.4600766283524904
Recal: 0.4444444444444444
f1_score: 0.4467338217338217
[[11 12]
 [18 13]]
RandomForestClassifier: 
Time: 0.05546736717224121
Acurácia: 0.42592592592592593
Precision: 0.4053497942386831
Recal: 0.42592592592592593
f1_score: 0.4123420836198412
[[ 5 18]
 [13 18]]
ExtraTreesClassifier: 
Time: 0.09374618530273438
Acurácia: 0.4444444444444444
Precision: 0.4600766283524904
Recal: 0.444

,ALGORITHM,TIME,ACCURACY,PRECISION,RECALL,F1_SCORE
0,MLPClassifier:,0.306464,0.481481,0.460637,0.481481,0.465468
1,AdaBoostClassifier:,0.074852,0.500000,0.484568,0.500000,0.488169
2,Voting_LR3_SVC1_ETC2:,0.236574,0.444444,0.371153,0.444444,0.391450
3,DecisionTreeClassifier:,0.003603,0.444444,0.460077,0.444444,0.446734
4,RandomForestClassifier:,0.055467,0.425926,0.405350,0.425926,0.412342
5,ExtraTreesClassifier:,0.093746,0.444444,0.460077,0.444444,0.446734
6,LogisticRegression:,0.009809,0.500000,0.418478,0.500000,0.427734
7,svm:,0.005024,0.574074,0.329561,0.574074,0.418736
8,Stacking_scikit:,1.007349,0.574074,0.329561,0.574074,0.418736
9,XGBClassifier:,0.065876,0.462963,0.452996,0.462963,0.456622


# Teste 4 - 'Idade', 'Peso Kg', 'Cor da pele',  'Score Força%', 'Score Resistência %'

In [103]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Idade', 'Peso Kg', 'Cor da pele', 'Score Força%', 'Score Resistência %']
col_to_be_encoded = ["Cor da pele", "Rabdomiólise"]
col_to_float = ['Idade', 'Peso Kg', 'Score Força%', 'Score Resistência %']

col_df = ['Idade', 'Peso Kg', 'Cor da pele', 'Score Força%', 'Score Resistência %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)
    
for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float: 
    df[d] = df[d].astype(float)

df

,Idade,Peso Kg,Cor da pele,Score Força%,Score Resistência %,Rabdomiólise
0,33.0,88.0,1,25.0,75.0,0
1,26.0,74.0,1,50.0,50.0,1
2,26.0,76.0,1,50.0,50.0,1
4,33.0,82.0,1,50.0,50.0,1
5,31.0,92.0,1,100.0,0.0,1
...,...,...,...,...,...,...
338,28.0,87.0,1,62.5,37.5,1
339,27.0,78.5,1,50.0,50.0,1
340,25.0,85.1,1,37.5,62.5,0
342,28.0,85.0,1,50.0,50.0,1


In [105]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label], 
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

MLPClassifier
Time: 0.2841067314147949
Acurácia: 0.48148148148148145
Precision: 0.4510921177587844
Recal: 0.48148148148148145
f1_score: 0.4565302144249512
[[ 5 18]
 [10 21]]
AdaBoostClassifier: 
Time: 0.07835054397583008
Acurácia: 0.48148148148148145
Precision: 0.4390938781182684
Recal: 0.48148148148148145
f1_score: 0.44547325102880664
[[ 4 19]
 [ 9 22]]
Voting_LR3_SVC1_ETC2: 
Time: 0.22416210174560547
Acurácia: 0.42592592592592593
Precision: 0.4053497942386831
Recal: 0.42592592592592593
f1_score: 0.4123420836198412
[[ 5 18]
 [13 18]]
DecisionTreeClassifier: 
Time: 0.0
Acurácia: 0.35185185185185186
Precision: 0.3679503367003367
Recal: 0.35185185185185186
f1_score: 0.34985070834127435
[[10 13]
 [22  9]]
RandomForestClassifier: 
Time: 0.062493085861206055
Acurácia: 0.4074074074074074
Precision: 0.4134355044699873
Recal: 0.4074074074074074
f1_score: 0.40987654320987654
[[ 8 15]
 [17 14]]
ExtraTreesClassifier: 
Time: 0.09971499443054199
Acurácia: 0.4074074074074074
Precision: 0.41838134430

,ALGORITHM,TIME,ACCURACY,PRECISION,RECALL,F1_SCORE
0,MLPClassifier:,0.284107,0.481481,0.451092,0.481481,0.456530
1,AdaBoostClassifier:,0.078351,0.481481,0.439094,0.481481,0.445473
2,Voting_LR3_SVC1_ETC2:,0.224162,0.425926,0.405350,0.425926,0.412342
3,DecisionTreeClassifier:,0.000000,0.351852,0.367950,0.351852,0.349851
4,RandomForestClassifier:,0.062493,0.407407,0.413436,0.407407,0.409877
5,ExtraTreesClassifier:,0.099715,0.407407,0.418381,0.407407,0.410677
6,LogisticRegression:,0.000000,0.555556,0.512327,0.555556,0.482621
7,svm:,0.004533,0.574074,0.329561,0.574074,0.418736
8,Stacking_scikit:,1.000981,0.574074,0.329561,0.574074,0.418736
9,XGBClassifier:,0.047007,0.425926,0.429012,0.425926,0.427328


# Teste 5 - 'Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)'

In [155]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)']

col_df = ['Pré Lactato (mmol/L)', 'Pós Lactato (mmol/L)', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)
    
for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float: 
    df[d] = df[d].astype(float)

df

,Pré Lactato (mmol/L),Pós Lactato (mmol/L),Rabdomiólise
0,0.0,0.0,0
1,0.0,0.0,1
2,0.0,0.0,1
3,0.0,0.0,1
4,0.0,0.0,1
...,...,...,...
423,1.7,2.2,1
424,2.1,2.2,1
425,2.6,1.8,1
426,2.5,1.9,1


In [157]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label], 
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

MLPClassifier
Time: 0.33280277252197266
Acurácia: 0.6835443037974683
Precision: 0.7455651631306829
Recal: 0.6835443037974683
f1_score: 0.6208122976551868
[[ 7 24]
 [ 1 47]]
AdaBoostClassifier: 
Time: 0.07813239097595215
Acurácia: 0.620253164556962
Precision: 0.5999196302993771
Recal: 0.620253164556962
f1_score: 0.5512229135378675
[[ 5 26]
 [ 4 44]]
Voting_LR3_SVC1_ETC2: 
Time: 0.25380659103393555
Acurácia: 0.6455696202531646
Precision: 0.6684749849306811
Recal: 0.6455696202531646
f1_score: 0.5690872751499001
[[ 5 26]
 [ 2 46]]
DecisionTreeClassifier: 
Time: 0.0
Acurácia: 0.6075949367088608
Precision: 0.5846648682299233
Recal: 0.6075949367088608
f1_score: 0.5789415057626813
[[ 9 22]
 [ 9 39]]
RandomForestClassifier: 
Time: 0.06293773651123047
Acurácia: 0.620253164556962
Precision: 0.5985232067510549
Recal: 0.620253164556962
f1_score: 0.5813350106140419
[[ 8 23]
 [ 7 41]]
ExtraTreesClassifier: 
Time: 0.09375619888305664
Acurácia: 0.6075949367088608
Precision: 0.5794547224926971
Recal: 0.

,ALGORITHM,TIME,ACCURACY,PRECISION,RECALL,F1_SCORE
0,MLPClassifier:,0.332803,0.683544,0.745565,0.683544,0.620812
1,AdaBoostClassifier:,0.078132,0.620253,0.599920,0.620253,0.551223
2,Voting_LR3_SVC1_ETC2:,0.253807,0.645570,0.668475,0.645570,0.569087
3,DecisionTreeClassifier:,0.000000,0.607595,0.584665,0.607595,0.578942
4,RandomForestClassifier:,0.062938,0.620253,0.598523,0.620253,0.581335
5,ExtraTreesClassifier:,0.093756,0.607595,0.579455,0.607595,0.562991
6,LogisticRegression:,0.016724,0.607595,0.369172,0.607595,0.459284
7,svm:,0.017612,0.645570,0.776149,0.645570,0.539644
8,Stacking_scikit:,0.979971,0.670886,0.786521,0.670886,0.587109
9,XGBClassifier:,0.057482,0.607595,0.581977,0.607595,0.571491


# Teste 6 - 'Pré LDH (U/L)',  'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L'

In [175]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré LDH (U/L)', 'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré LDH (U/L)', 'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
                'Pré Albumina (g/dL)',
                'Pós Albumina (g/dL)', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)', 'Pós TFG   (ml/min/1.73 m2)',
                'Pré Ureia mg/dL', 'Pós Ureia mg/dL', 'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
               'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
                'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
                'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L', 
                'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L']

col_df = ['Pré LDH (U/L)', 'Pós LDH (U/L)', 'Pré AST (U/L)', 'Pós AST (U/L)',
       'Pré Albumina (g/dL)', 'Pós Albumina (g/dL)', 'Pré TGP (g/dL)',
       'Pós TGP (g/dL)', 'Pré GGT U/L', 'Pós GGT U/L', 'Pré Ac. Úrico (mg/dL)',
       'Pós Ac. Úrico (mg/dL)', 'Pré TFG   (ml/min/1.73 m2)',
       'Pós TFG   (ml/min/1.73 m2)', 'Pré Ureia mg/dL', 'Pós Ureia mg/dL',
       'Pré creatinina mg/dL', 'Pós creatinina mg/dL',
       'Delta CREATININA mg/dL', 'Pré Fósforo mg/dL', 'Pós Fósforo mg/dL',
       'Pré Proteína Total (g/dL)', 'Pós Proteína Total (g/dL)',
       'Pré FE (ug/dL)', 'Pós FE (ug/dL)', 'Pré calcio mg/dL',
       'Pós calcio mg/dL', 'Pré Potássio mmol/L', 'Pós Potássio mmol/L',
       'Glicose pré', 'Glicose pós', 'Pré Sódio mmol/L', 'Pós Sódio mmol/L',
       'Pré Magnésio mg/dL', 'Pós Magnésio mg/dL', 'Pré Cloro mmol/L',
       'Pós Cloro mmol/L', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)
    
for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float: 
    df[d] = df[d].astype(float)

df

,Pré LDH (U/L),Pós LDH (U/L),Pré AST (U/L),Pós AST (U/L),Pré Albumina (g/dL),Pós Albumina (g/dL),Pré TGP (g/dL),Pós TGP (g/dL),Pré GGT U/L,Pós GGT U/L,...,Pós Potássio mmol/L,Glicose pré,Glicose pós,Pré Sódio mmol/L,Pós Sódio mmol/L,Pré Magnésio mg/dL,Pós Magnésio mg/dL,Pré Cloro mmol/L,Pós Cloro mmol/L,Rabdomiólise
0,629.0,796.0,33.0,48.0,4.6,4.0,37,63.0,16,17.0,...,4.6,103,110.0,145.0,146.0,2.0,2.0,103.0,106.0,0
1,720.0,1182.0,49.0,105.0,4.9,4.6,57,93.0,38,37.0,...,4.3,95,114.0,142.0,141.0,2.2,1.9,102.0,103.0,1
2,895.0,1531.0,57.0,145.0,4.9,4.8,58,113.0,26,23.0,...,4.5,99,98.0,145.0,146.0,2.2,2.2,105.0,100.0,1
3,349.0,666.0,46.0,138.0,4.8,4.1,42,100.0,13,10.0,...,4.6,94,144.0,144.0,143.0,2.1,2.0,106.0,108.0,1
4,643.0,1089.0,32.0,94.0,4.8,4.3,32,85.0,39,40.0,...,4.5,96,142.0,142.0,142.0,2.0,1.9,102.0,103.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,191.0,642.0,36.0,116.0,4.4,4.7,38,42.0,17,19.0,...,4.3,85,91.0,139.0,137.0,2.0,2.0,104.0,95.0,1
340,190.0,751.0,29.0,59.0,4.4,5.1,25,28.0,19,22.0,...,4.7,91,94.0,139.0,138.0,2.0,2.3,101.0,95.0,0
341,228.0,461.0,30.0,57.0,4.4,5.1,29,43.0,25,30.0,...,4.4,88,101.0,140.0,138.0,2.1,2.0,103.0,95.0,0
342,246.0,545.0,31.0,82.0,4.5,5.5,28,33.0,17,18.0,...,5.0,93,107.0,138.0,139.0,2.1,2.3,101.0,96.0,1


In [177]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label], 
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

MLPClassifier
Time: 0.050191402435302734
Acurácia: 0.4927536231884058
Precision: 0.495351380913317
Recal: 0.4927536231884058
f1_score: 0.4536555251785872
[[ 8 27]
 [ 8 26]]
AdaBoostClassifier: 
Time: 0.14226889610290527
Acurácia: 0.8405797101449275
Precision: 0.8481976960237829
Recal: 0.8405797101449275
f1_score: 0.8399077595027566
[[27  8]
 [ 3 31]]
Voting_LR3_SVC1_ETC2: 
Time: 0.2636148929595947
Acurácia: 0.7681159420289855
Precision: 0.7721512758052212
Recal: 0.7681159420289855
f1_score: 0.7675303762260284
[[25 10]
 [ 6 28]]
DecisionTreeClassifier: 
Time: 0.0
Acurácia: 0.8115942028985508
Precision: 0.8186547751765143
Recal: 0.8115942028985508
f1_score: 0.8108000794123487
[[26  9]
 [ 4 30]]
RandomForestClassifier: 
Time: 0.07376861572265625
Acurácia: 0.8115942028985508
Precision: 0.8186547751765143
Recal: 0.8115942028985508
f1_score: 0.8108000794123487
[[26  9]
 [ 4 30]]
ExtraTreesClassifier: 
Time: 0.11790943145751953
Acurácia: 0.8405797101449275
Precision: 0.8481976960237829
Recal:

,ALGORITHM,TIME,ACCURACY,PRECISION,RECALL,F1_SCORE
0,MLPClassifier:,0.050191,0.492754,0.495351,0.492754,0.453656
1,AdaBoostClassifier:,0.142269,0.840580,0.848198,0.840580,0.839908
2,Voting_LR3_SVC1_ETC2:,0.263615,0.768116,0.772151,0.768116,0.767530
3,DecisionTreeClassifier:,0.000000,0.811594,0.818655,0.811594,0.810800
4,RandomForestClassifier:,0.073769,0.811594,0.818655,0.811594,0.810800
5,ExtraTreesClassifier:,0.117909,0.840580,0.848198,0.840580,0.839908
6,LogisticRegression:,0.009115,0.768116,0.769214,0.768116,0.768019
7,svm:,0.015489,0.826087,0.826087,0.826087,0.826087
8,Stacking_scikit:,1.270314,0.797101,0.806847,0.797101,0.795816
9,OvR_RF:,0.083023,0.811594,0.818655,0.811594,0.810800


# Teste 7 - Série vermelha -  'Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %'

In [179]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %']

col_df = ['Pré RCB 10^6/mm³', 'Pós RCB 10^6/mm³', 'Pré HGB g/dL', 'Pós HGB g/dL',
       'Pré HCT %', 'Pós HCT %', 'Pré MCV fL', 'Pós MCV fL', 'Pré MCH pg',
       'Pós MCH pg', 'Pré MCHC g/dL', 'Pós MCHC g/dL', 'Pré PLT 10³/mm³',
       'Pós PLT 10³/mm³', 'Pré RDW %', 'Pós RDW %', 'Pré RET %', 'Pós RET %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)
    
for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float: 
    df[d] = df[d].astype(float)

df

,Pré RCB 10^6/mm³,Pós RCB 10^6/mm³,Pré HGB g/dL,Pós HGB g/dL,Pré HCT %,Pós HCT %,Pré MCV fL,Pós MCV fL,Pré MCH pg,Pós MCH pg,Pré MCHC g/dL,Pós MCHC g/dL,Pré PLT 10³/mm³,Pós PLT 10³/mm³,Pré RDW %,Pós RDW %,Pré RET %,Pós RET %,Rabdomiólise
0,4.38,4.37,13.5,13.3,40.9,39.4,93.4,90.2,30.8,30.4,33.0,33.8,299.0,289.0,12.8,12.2,1.83,1.52,0
1,0.00,4.64,0.0,13.9,0.0,40.6,0.0,87.5,0.0,30.0,0.0,34.2,268.0,304.0,0.0,13.4,0.00,1.47,1
2,0.00,4.77,0.0,14.5,0.0,42.7,0.0,89.5,0.0,30.4,0.0,34.0,0.0,268.0,0.0,12.1,0.00,1.20,1
3,5.22,4.52,14.9,13.1,47.7,38.7,91.4,85.6,28.5,29.0,31.2,33.9,242.0,246.0,12.9,12.4,0.00,1.11,1
4,4.37,4.06,13.3,12.5,41.9,37.5,95.9,92.4,30.4,30.8,31.7,33.3,192.0,259.0,11.7,12.1,0.00,2.27,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,4.58,4.48,14.4,16.2,44.2,48.4,96.4,95.4,31.4,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.56,1
340,4.49,5.61,13.7,11.8,42.2,36.5,93.8,93.7,30.4,30.3,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.10,0
341,4.61,4.70,14.0,12.5,42.7,37.9,92.5,92.0,30.3,30.3,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.96,0
342,4.64,3.24,13.8,12.3,41.7,37.3,89.8,89.6,29.8,29.5,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.57,1


In [181]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label], 
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

MLPClassifier
Time: 0.33229827880859375
Acurácia: 0.5217391304347826
Precision: 0.5242474916387959
Recal: 0.5217391304347826
f1_score: 0.48875562218890556
[[ 9 25]
 [ 8 27]]
AdaBoostClassifier: 
Time: 0.09375858306884766
Acurácia: 0.6376811594202898
Precision: 0.6376445615576051
Recal: 0.6376811594202898
f1_score: 0.6375288286065082
[[21 13]
 [12 23]]
Voting_LR3_SVC1_ETC2: 
Time: 0.2811148166656494
Acurácia: 0.6666666666666666
Precision: 0.7035087719298245
Recal: 0.6666666666666666
f1_score: 0.6489086200517943
[[15 19]
 [ 4 31]]
DecisionTreeClassifier: 
Time: 0.0
Acurácia: 0.5942028985507246
Precision: 0.5941784175479827
Recal: 0.5942028985507246
f1_score: 0.5936905284731372
[[19 15]
 [13 22]]
RandomForestClassifier: 
Time: 0.0888814926147461
Acurácia: 0.6956521739130435
Precision: 0.6992128935532234
Recal: 0.6956521739130435
f1_score: 0.6937198067632849
[[21 13]
 [ 8 27]]
ExtraTreesClassifier: 
Time: 0.10850381851196289
Acurácia: 0.6811594202898551
Precision: 0.7045384464440894
Recal:

,ALGORITHM,TIME,ACCURACY,PRECISION,RECALL,F1_SCORE
0,MLPClassifier:,0.332298,0.521739,0.524247,0.521739,0.488756
1,AdaBoostClassifier:,0.093759,0.637681,0.637645,0.637681,0.637529
2,Voting_LR3_SVC1_ETC2:,0.281115,0.666667,0.703509,0.666667,0.648909
3,DecisionTreeClassifier:,0.000000,0.594203,0.594178,0.594203,0.593691
4,RandomForestClassifier:,0.088881,0.695652,0.699213,0.695652,0.693720
5,ExtraTreesClassifier:,0.108504,0.681159,0.704538,0.681159,0.670328
6,LogisticRegression:,0.015211,0.579710,0.595271,0.579710,0.557320
7,svm:,0.004414,0.695652,0.756689,0.695652,0.674663
8,Stacking_scikit:,1.215113,0.710145,0.766221,0.710145,0.692515
9,OvR_RF:,0.079034,0.695652,0.699213,0.695652,0.693720


# Teste 8 - Série leucocitária - 'Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %'

In [190]:
col_label = ["Rabdomiólise"]
label_lesao = ["Lesão Renal Aguda"]
col_feature = ['Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %']
col_to_be_encoded = ["Rabdomiólise"]
col_to_float = ['Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %']

col_df = ['Pré WBC 10³/mm³', 'Pós WBC 10³/mm³', 'Pré NEUT %', 'Pós NEUT %',
       'Pré LINF %', 'Pós LINF %', 'Pré MONO %', 'Pós MONO %', 'Pré EOS %',
       'Pós EOS %', 'Pré BASO %', 'Pós BASO %', "Rabdomiólise" ]

df = DATASET[col_df].dropna()
encoder = preprocessing.LabelEncoder()
for c in col_df:
    df = df.drop(df[df[c] == '-'].index)
    
for c in col_to_be_encoded:
    df[c] = encoder.fit_transform(df[c])

# convert to float
for d in col_to_float: 
    df[d] = df[d].astype(float)

df

,Pré WBC 10³/mm³,Pós WBC 10³/mm³,Pré NEUT %,Pós NEUT %,Pré LINF %,Pós LINF %,Pré MONO %,Pós MONO %,Pré EOS %,Pós EOS %,Pré BASO %,Pós BASO %,Rabdomiólise
0,7.38,8.17,49.1,72.6,39.0,21.1,8.3,5.3,3.3,0.9,0.3,0.1,0
1,0.00,9.38,0.0,72.0,0.0,12.5,0.0,15.0,0.0,0.3,0.0,0.2,1
2,0.00,7.48,0.0,82.8,0.0,11.2,0.0,5.6,0.0,0.1,0.0,0.3,1
3,4.91,5.79,52.0,67.8,34.6,21.6,11.2,6.2,1.2,3.5,1.0,0.9,1
4,6.31,10.56,79.1,82.6,15.2,10.9,4.1,5.5,1.1,0.9,0.5,0.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,6.63,8.28,59.1,55.6,30.3,34.1,8.0,8.5,1.4,1.0,1.2,0.8,0
443,8.61,8.28,59.5,70.7,31.5,20.6,6.7,8.3,2.0,0.1,0.3,0.3,0
444,5.45,15.77,44.6,52.3,41.8,36.3,9.0,10.4,3.9,0.6,0.7,0.4,0
445,4.90,11.69,57.4,57.5,31.0,31.4,8.8,9.9,2.0,0.7,0.8,0.5,0


In [192]:
X_train, X_test, y_train, y_test = train_test_split( df[col_feature],
                                                    df[col_label], 
                                                    train_size=0.80, random_state=28)

results = get_tests_result(X_train, X_test, y_train, y_test)
results

MLPClassifier
Time: 0.31292152404785156
Acurácia: 0.6626506024096386
Precision: 0.6613778041337016
Recal: 0.6626506024096386
f1_score: 0.6604574325671481
[[22 16]
 [12 33]]
AdaBoostClassifier: 
Time: 0.09213900566101074
Acurácia: 0.6626506024096386
Precision: 0.6616394656640747
Recal: 0.6626506024096386
f1_score: 0.6617586680446818
[[23 15]
 [13 32]]
Voting_LR3_SVC1_ETC2: 
Time: 0.3793609142303467
Acurácia: 0.6987951807228916
Precision: 0.7125135022850021
Recal: 0.6987951807228916
f1_score: 0.686726564139127
[[19 19]
 [ 6 39]]
DecisionTreeClassifier: 
Time: 0.00597071647644043
Acurácia: 0.6626506024096386
Precision: 0.6618975903614458
Recal: 0.6626506024096386
f1_score: 0.6587349397590361
[[21 17]
 [11 34]]
RandomForestClassifier: 
Time: 0.07906627655029297
Acurácia: 0.6867469879518072
Precision: 0.6859080981790306
Recal: 0.6867469879518072
f1_score: 0.684710473098066
[[23 15]
 [11 34]]
ExtraTreesClassifier: 
Time: 0.12061142921447754
Acurácia: 0.7228915662650602
Precision: 0.723329682

,ALGORITHM,TIME,ACCURACY,PRECISION,RECALL,F1_SCORE
0,MLPClassifier:,0.312922,0.662651,0.661378,0.662651,0.660457
1,AdaBoostClassifier:,0.092139,0.662651,0.661639,0.662651,0.661759
2,Voting_LR3_SVC1_ETC2:,0.379361,0.698795,0.712514,0.698795,0.686727
3,DecisionTreeClassifier:,0.005971,0.662651,0.661898,0.662651,0.658735
4,RandomForestClassifier:,0.079066,0.686747,0.685908,0.686747,0.684710
5,ExtraTreesClassifier:,0.120611,0.722892,0.723330,0.722892,0.720427
6,LogisticRegression:,0.005166,0.698795,0.697978,0.698795,0.697464
7,svm:,0.013335,0.590361,0.632415,0.590361,0.519485
8,Stacking_scikit:,1.217205,0.722892,0.733673,0.722892,0.714534
9,OvR_RF:,0.078376,0.686747,0.685908,0.686747,0.684710
